# Image Filtering
---
|TABLE|OF|CONTENT|||||||||||||||||||||||||||||||||||||||||||||||||||
|---|||||||||||||||||||||||||||||||||||||||||||||||||||||
[Introduction](#intro)
[Convolution](#conv)
[Sharpen](#sharpen)
[Blur](#blur)
[Emboss](#emboss)
[Motion Blur](#motion_blur)
[Find Edges](#edge_detect)
[Sepia](#sepia)

<div id='intro'></div>

### Introduction
---
- *Image filtering allows us to apply various effects on photos.*
- *The type of image filtering described here uses a 2D filter similar to the one included in Paint Shop*
*Pro as User Defined Filter and in Photoshop as Custom Filter.*

<div id='conv'></div>

### Convolution
---
- *The trick of image filtering is that you have a 2D filter matrix, and the 2D image.*
- *Then, for every pixel of the image, take the sum of products. Each product is the color value of the current pixel or a neighbor of it, with the corresponding value of the filter matrix.*
- *The center of the filter matrix( **anchor of the filter** ) has to be multiplied with the current pixel, the other elements of the filter matrix with corresponding neighbor pixels.*

#### *Rules about the filters:*
- Size is preferred to be of odd dimension so it's centered and padding can done in case of same convolution ($padding = \frac{f-1}{2}$ for same convolution).
- Sum of the elements of the filter is preferred to be one for output image to be of same brightness as input image.
- If the sum of the elements is larger than 1, the result will be a brighter image, and if it's smaller than 1, a darker image.

In [43]:
import cv2
import numpy as np
import scipy
import copy

In [44]:
def display(image):
    window_name='scenery'
    # print(image.shape) (400,400,3)
    cv2.namedWindow(window_name,cv2.WINDOW_NORMAL)
    cv2.imshow(window_name,image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [45]:
def write(text,image):
    cv2.imwrite(text+".jpeg",image)

### Original Image  
---
![Scenery](images/scenery.jpeg)

In [47]:
path=r'scenery.jpeg'
image=cv2.imread(path)
display(image)

In [36]:
def apply(image,kernel):
    _,f=kernel.shape
    p=int((f-1)//2)
    image=np.pad(image,((p,p),(p,p),(0,0)))
    nh,nw,nc=image.shape
    result=np.zeros((nh-f+1,nw-f+1,nc))
    for c in range(nc):
        for w in range(nw):
            for h in range(nh):
                if h+f-1>=nh or w+f-1>=nw:
                    break
                result[h][w][c]=min(max(int(np.sum(np.multiply(image[h:h+f,w:w+f,c],kernel))),0),255)
    return result

<div id='sharpen'></div>

# Sharpen Image
---
![Sharpen](images/sharpen.jpeg)

In [42]:
def sharpen(img):
    image=copy.deepcopy(img)
    kernel=np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
#     return apply(image,kernel)
    return cv2.filter2D(image,-1,kernel)

<div id='blur'></div>

# Blur Image
---
* Blurring is done for example by taking the average of the current pixel and its 4 neighbors. 
* Take the sum of the current pixel and its 4 neighbors, and divide it through 5, or thus fill in 5 times the value 0.2 in the filter:

![Blur](images/blur.jpeg)

In [38]:
def blur(img):
    image=copy.deepcopy(img)
    kernel=np.array([[0.0,0.2,0.0],[0.2,0.2,0.2],[0.0,0.2,0.0]])
    return cv2.filter2D(image,-1,kernel)
#     return apply(image,kernel)
display(blur(image))

<div id='emboss'></div>

# Emboss Image
---
- An emboss filter gives a 3D shadow effect to the image, the result is very useful for a bumpmap of the image. 
- It can be achieved by taking a pixel on one side of the center, and subtracting one of the other side from it.
- Pixels can get either a positive or a negative result. 
- To use the negative pixels as shadow and positive ones as light, for a bumpmap, a bias of 128 is added to the image. 
- Now, most parts of the image will be gray, and the sides will be either dark gray/black or bright gray/white.

For example here's an emboss filter with an angle of 45°:

![Emboss](images/emboss.jpeg)

In [39]:
def emboss(img):
    image=copy.deepcopy(img)
    kernel=np.array([[-1,-1,0],[-1,0,1],[0,1,1]])
    return cv2.filter2D(image,-1,kernel)
#     return apply(image,kernel)
display(emboss(image))

<div id='motion_blur'></div>

# Motion Blur Image
- Motion blur is achieved by blurring in only 1 direction
- It's as if the camera is moving from the top left to the bottom right, hence the name.

![Motion Blur](images/motion_blur.jpeg)

In [40]:
def motion_blur(img):
    image=copy.deepcopy(img)
    kernel=(1/9)*np.eye(9)
    return cv2.filter2D(image,-1,kernel)
display(motion_blur(image))

<div id='edge_detect'></div>

# Edges Detection Image
---
- A filter to find the diagonal edges looks like this:
![edges_detection](images/edges_detection.jpeg)

In [50]:
def edges_detection(img):
    image=copy.deepcopy(img)
    kernel=np.array([
        [-1,  0, 0,  0,  0],
        [0,  -2, 0,  0,  0],
        [0,  0,  6,  0,  0],
        [0,  0,  0,  -2,  0],
        [0,  0,  0,  0,  -1]])
    return cv2.filter2D(image,-1,kernel)
display(edges_detection(image))

<div id='sepia'></div>

### Sepia Image
---
- The sepia effect is a well-known tonal editing technique that adds a warmer tone that gives an image a vintage or archival quality. 
- If you want to add a bit of style to your composition, the sepia effect works exceptionally well with black and white images.
![sepia](images/sepia.jpeg)

In [79]:
def sepia(img):
    image=copy.deepcopy(img)
#     kernel=np.random.rand(3,3)
#     Following is a special sepia matrix used often.
    kernel = np.array([[0.272, 0.534, 0.131],
                       [0.349, 0.686, 0.168],
                       [0.393, 0.769, 0.189]])
    return cv2.filter2D(image, -1, kernel)
display(sepia(image))